# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

**Load libraries**

In [105]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from math import log

**Load the house sales data**

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [106]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int,
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float,
              'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int,
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
train_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [107]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4229 entries, 0 to 4228
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4229 non-null   object 
 1   date           4229 non-null   object 
 2   price          4229 non-null   float64
 3   bedrooms       4229 non-null   float64
 4   bathrooms      4229 non-null   float64
 5   sqft_living    4229 non-null   float64
 6   sqft_lot       4229 non-null   int64  
 7   floors         4229 non-null   object 
 8   waterfront     4229 non-null   int64  
 9   view           4229 non-null   int64  
 10  condition      4229 non-null   int64  
 11  grade          4229 non-null   int64  
 12  sqft_above     4229 non-null   int64  
 13  sqft_basement  4229 non-null   int64  
 14  yr_built       4229 non-null   int64  
 15  yr_renovated   4229 non-null   int64  
 16  zipcode        4229 non-null   object 
 17  lat            4229 non-null   float64
 18  long    

**Train a multiple regression model**

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [108]:
example_model = smf.ols('price ~ sqft_living + bedrooms + bathrooms', data = train_data).fit()

Now that we have fitted the model we can extract the regression weights (coefficients):

In [109]:
example_model.summary()
example_model.params

Intercept      87912.865815
sqft_living      315.406691
bedrooms      -65081.887116
bathrooms       6942.165986
dtype: float64

**Make in-sample predictions**

In the gradient descent notebook we use numpy to do our regression.

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [110]:
example_predictions = example_model.predict(train_data)
print(example_predictions[0])

271789.2653799552


**Compute the RSS**

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [111]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    preds = model.predict(data)
    # Then compute the residuals/errors
    errors = outcome - preds
    # Then square and add them up
    RSS = (errors**2).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [112]:
rss_example_test = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print(rss_example_test)

273761940583134.44


**Create new features**

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the square feet or even "interaction" features such as the product of bedrooms and bathrooms.

Create the following 4 new features as columns in both the train and test data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

In [113]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

train_data['bed_bath_rooms'] = train_data.apply(lambda x: x['bedrooms'] * x['bathrooms'], axis = 1)
test_data['bed_bath_rooms'] = test_data.apply(lambda x: x['bedrooms'] * x['bathrooms'], axis = 1)

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data.apply(lambda x: x['lat'] + x['long'], axis = 1)
test_data['lat_plus_long'] = test_data.apply(lambda x: x['lat'] + x['long'], axis = 1)

train_data[['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']].head()

,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,9.0,3.00,7.073270,-74.7458
1,9.0,6.75,7.851661,-74.5980
2,4.0,2.00,6.646391,-74.4951
3,16.0,12.00,7.580700,-74.8722
4,9.0,6.00,7.426549,-74.4282


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16, which means this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally nonsensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on the test data? (round to 2 digits)**

In [114]:
new_feature_mean = test_data[['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']].mean()
new_feature_mean.round(2)

bedrooms_squared    12.45
bed_bath_rooms       7.50
log_sqft_living      7.55
lat_plus_long      -74.65
dtype: float64

**Training multiple models**

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [115]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, train the weights for the three different models for predicting target = 'price' and look at the value of the weights/coefficients:

In [116]:
# train model 1
model_1 = smf.ols('price ~ sqft_living + bedrooms + bathrooms + lat + long', data = train_data).fit()

# train model 2
model_2 = smf.ols('price ~ sqft_living + bedrooms + bathrooms + lat + long + bed_bath_rooms', data = train_data).fit()

# train model 3
model_3 = smf.ols('price ~ sqft_living + bedrooms + bathrooms + lat + long + bed_bath_rooms \
                  + bedrooms_squared + log_sqft_living + lat_plus_long', data = train_data).fit()

In [117]:
# Examine/extract each model's coefficients:
for model in [model_1, model_2, model_3]:
    print(model.params)
    print('-----')

Intercept     -6.907573e+07
sqft_living    3.122586e+02
bedrooms      -5.958653e+04
bathrooms      1.570674e+04
lat            6.586193e+05
long          -3.093744e+05
dtype: float64
-----
Intercept        -6.686797e+07
sqft_living       3.066101e+02
bedrooms         -1.134464e+05
bathrooms        -7.146131e+04
lat               6.548446e+05
long             -2.942990e+05
bed_bath_rooms    2.557965e+04
dtype: float64
-----
Intercept          -6.203608e+07
sqft_living         5.294228e+02
bedrooms            3.451423e+04
bathrooms           6.706078e+04
lat                 5.340856e+05
long               -4.067507e+05
bed_bath_rooms     -8.570504e+03
bedrooms_squared   -6.788587e+03
log_sqft_living    -5.618315e+05
lat_plus_long       1.273349e+05
dtype: float64
-----


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

Positive

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Negative

Think about what this means.
It means that there is a significant interaction effect/multicollinearity with at least one another variable.

**Comparing different models**

Now that you've trained three models and extracted the model weights, we want to evaluate which model is best.

Use your functions from earlier to compute the RSS on the training data for each of the three models.

In [118]:
# Compute the RSS on the training data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print(rss_model_1)
print(rss_model_2)
print(rss_model_3)

967879963049545.6
958419635074070.4
903436455050478.1


**Quiz Question: Which model (1, 2 or 3) has the lowest RSS on the training data?** Is this what you expected?

The model with the most parameters. Yes, this is not unexpected and a common result.

Now compute the RSS on on TEST data for each of the three models.

In [119]:
# Compute the RSS on the test data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print(rss_model_1)
print(rss_model_2)
print(rss_model_3)

225500469795490.56
223377462976467.06
259236319207174.56


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on the test data?** Is this what you expected? Think about the features that were added to each model from the previous.

The model with the middling amount of parameters (model 2). It indicates that the extra variables in model 3 do not provide significant additional information about house prices.